# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `os` module provides functions to perform tasks such as file and directory operations, process management, and environment variable manipulation.
* The `sys` module facilitates interaction with and manipulation of the Python runtime environment's system-specific parameters and functions.
* The `json` module allows developers to load, read and write JSON files.
* The `math` module provides a collection of mathematical functions and constants for performing mathematical operations.
* The `yaml` module allows developers to load, read and write YAML files.
* The `sys` module contains methods and variables for modifying many elements of the Python runtime environment.
* The `tabulate` module allows the user to display data in a table format.
* The `google.colab` module provides access to some of the unique features and functionality of Google Colab.

In [ ]:
import ee
import os
import sys
import json
import math
import yaml
import datetime
import tabulate

from google.colab import drive

# **2) Authentication Procedure**

This section provides instructions for setting up the Google Earth Engine Python API on Colab and for setting up Google Drive on Colab. These steps should be performed each time you start/restart/rollback a Colab session.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers, while the `ee.Initialize` function initializes it. After executing the following cell, the user is prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory.

In [ ]:
ee.Authenticate()
ee.Initialize(project="...")

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-o_7Qxd5sqGOFBEaBJ1IuLliyssviIiu-fnxhQLVNSc&tc=p6wXQOt2JZFRf6p6mz5AArpVyajhOTmuLwOqxLw_uMo&cc=rLndFZsQFJpfNCSNmuoZN6LOO7-Af4TyYAqmmNGqXAE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmoW7bF1QhSp51gxWTdVrEfrb5oAXRSh_h6Y57MRmsRahSwVuEiFNI

Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


## **2.2) GD**

The `drive.mount` function allows access to specific folders of Google Drive. Granting access to Google Drive allows code running in the notebook to modify files in Google Drive.

**Note:** When using the `Mount Drive` button in the file browser, no authentication codes are required for notebooks edited only by the current user.

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# **3) Functions**

Data Processing

In [ ]:
sys.path.append(".../utilities")
import S1

In [ ]:
def export_tasks_viewer(exportTasksIds, tableFormat: str = "plain"):
  """
  Description:
    Displays a table view which contains useful information about the provided export tasks.

  Notes:
    * Task_Id: The task identifier.
    * Task_State: One of READY, RUNNING, COMPLETED, FAILED, CANCELLED, UNSUBMITTED or UNKNOWN.
    * Task_Type: One of EXPORT_IMAGE, EXPORT_TILES, EXPORT_FEATURES, EXPORT_VIDEO.
    * Task_Attempt: Number of attempts.
    * Task_Description: A human-readable description of the task.
    * Queue_Time: The time that is taken while being in a queue.
    * Execution_Time: The time spent by the servers executing the task.
    * Completion_Time: SUm of queue and execution times.
    * Error_Message: Failure reason. Appears only if state is FAILED. May also include other fields.

  Arguments:
    exportTasksIdsList (list) (mandatory) A list of export task identifiers.
    tableFormat (str) (optional) The table format to use. Defaults to "plain".

  Returns:
    None, displays the export tasks table.
  """
  taskInfo = []
  tableHeaders = [
    "Task_Id", "Task_State", "Task_Type", "Task_Attempt", "Task_Description",
    "Queue_Time", "Execution_Time", "Completion_Time", "Error_Message"
  ]
  tableFormats = tabulate._table_formats.keys()

  if tableFormat not in tableFormats:
    raise ValueError(f"Invalid table format. Choose from: `{tableFormats}`.")

  # Populate taskInfo.
  for exportTaskId in exportTasksIds:

    taskState = ee.data.getTaskStatus(exportTaskId)[0]["state"]
    taskType = ee.data.getTaskStatus(exportTaskId)[0]["task_type"]
    taskDescription = ee.data.getTaskStatus(exportTaskId)[0]["description"]
    startTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["start_timestamp_ms"]/1000.0)
    updateTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["update_timestamp_ms"]/1000.0)
    creationTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["creation_timestamp_ms"]/1000.0)

    queueTime = None
    taskAttempt = None
    executionTime = None
    completionTime = None

    if taskState not in ["READY", "RUNNING"]:
      queueTime = (startTimestamp - creationTimestamp).total_seconds()
      executionTime = (updateTimestamp - startTimestamp).total_seconds()

    if taskState == "COMPLETED":
      taskAttempt = ee.data.getTaskStatus(exportTaskId)[0]["attempt"]
      completionTime = (updateTimestamp - creationTimestamp).total_seconds()

    try:
      errorMessage = ee.data.getTaskStatus(exportTaskId)[0]["error_message"]
    except KeyError:
      errorMessage = None  # This just means that the export task has not failed.

    taskInfo.append([exportTaskId, taskState, taskType, taskAttempt, taskDescription, queueTime, executionTime, completionTime, errorMessage])

  # Table display.
  table = tabulate.tabulate(taskInfo, headers=tableHeaders, tablefmt=tableFormat)
  print(table)

# **4) Parameters**

In [ ]:
# `Sentinel-1 GRD`
b1 = "VV"
vh = "VH"

# `Digital Elevation Model`
demProvider = "USGS"

# EMS case of interest
caseCode = "emsr692"
caseArea = "magnesia"

# Projection of interest
projectionCRS = "EPSG:4326"
projectionScale = 10

# `Classification`
rasterIdentifier = "050224_060B99_3635_050224_060B99_D80F"
preEventRasterIdentifier = "S1A_IW_GRDH_1SDV_20230907T162437_20230907T162502_050224_060B99_3635"
postEventRasterIdentifier = "S1A_IW_GRDH_1SDV_20230907T162412_20230907T162437_050224_060B99_D80F"

classifierFeatures = [
  "VHVHD", "VHVHQ", "VVVHD", "VVVHQ", "VVVVD", "VVVVQ", "NDPID", "NDPIQ",
  "PRE_VV", "PRE_VH", "PRE_NDPI", "POST_VV", "POST_VH", "POST_NDPI"
]

# GEE paths
classifierIdentifier = "..."

destinationFolder = "..."

# GD paths
configFile = "..."

# **5) Configuration**

In [ ]:
# `Digital Elevation Models`
demConfigs = {
  "CGIAR": {    # `SRTM Digital Elevation Data Version 4`
    "name": "CGIAR/SRTM90_V4"
  },
  "NASA": {     # `NASA NASADEM Digital Elevation`
    "name": "NASA/NASADEM_HGT/001"
  },
  "USGS": {     # `NASA SRTM Digital Elevation`
    "name": "USGS/SRTMGL1_003"
  },
  "ASTER": {    # AG100: ASTER Global Emissivity Dataset 100-meter V003`
    "name": "NASA/ASTER_GED/AG100_003"
  }
}

# `Sentinel-1 GRD`
s1Config = {
  "name": "COPERNICUS/S1_GRD"
}

bandCombinations = {
  "sum": {
    "expression": "b1 + b2",
    "name": "b1b2S"
  },
  "difference": {
    "expression": "b1 - b2",
    "name": "b1b2D"
  },
  "product": {
    "expression": "b1 * b2",
    "name": "b1b2P"
  },
  "quotient": {
    "expression": "b1 / b2",
    "name": "b1b2Q"
  },
  "ndpi": {
    "expression": "(b1 - b2) / (b1 + b2)",
    "name": "NDPI"
  }
}

In [ ]:
# Parse the appropriate configuration.
extension = os.path.splitext(configFile)[-1]

try:
  with open(configFile, "r") as stream:
    # JSON
    if extension == ".json":
      caseConfigs = json.load(stream)
    # YAML
    elif extension in (".yaml", ".yml"):
      caseConfigs = yaml.safe_load(stream)
    else:
      raise ValueError(f"Unsupported file format `{extension}`. Supported formats: `JSON`, `YAML`")

except FileNotFoundError as e:
  print(f"Error: JSON file not found: {e}")

# GEE assets
demConfig = demConfigs[demProvider]

caseConfig = caseConfigs[caseCode][caseArea]
caseConfig["area_of_interest"] = ee.FeatureCollection(caseConfig["area_of_interest"])

# **6) Data Processing**

Define the projection of interest.

In [ ]:
projection = ee.Projection(projectionCRS).atScale(projectionScale)

Load, filter and process raster collections.

In [ ]:
# `Digital Elevation Elevation`
elevation = ee.Image(demConfig["name"]).unmask()
slope = ee.Terrain.slope(elevation)

# `Sentinel-1 GRD`
preEventRaster = ee.Image("/".join([s1Config["name"], preEventRasterIdentifier]))
postEventRaster = ee.Image("/".join([s1Config["name"], postEventRasterIdentifier]))

preEventRaster = preEventRaster.addBands(**{
  "srcImg": preEventRaster.select("angle").updateMask(preEventRaster.select("angle").mask().gt(0)),
  "overwrite": True,
})

postEventRaster = postEventRaster.addBands(**{
  "srcImg": postEventRaster.select("angle").updateMask(postEventRaster.select("angle").mask().gt(0)),
  "overwrite": True,
})

acquisition = postEventRaster.date()
eventDate = ee.Date(caseConfig["event_date"])

# Apply angular-based radiometric slope correction.
preEventRaster = slope_correction(preEventRaster, elevation)
postEventRaster = slope_correction(postEventRaster, elevation)

# Apply a Refined-Lee speckle speckle noise filter.
preEventRaster = ee.Image(refined_lee(preEventRaster))    \
  .select(["VV", "VH"])                                   \
  .clipToCollection(caseConfig["area_of_interest"])       \
  .reproject(projection)

postEventRaster = ee.Image(refined_lee(postEventRaster))  \
  .select(["VV", "VH"])                                   \
  .clipToCollection(caseConfig["area_of_interest"])       \
  .reproject(projection)

Engineer new raster.

In [ ]:
differenceExpression = bandCombinations["difference"]["expression"]
quotientExpression = bandCombinations["quotient"]["expression"]
ndpiExpression = bandCombinations["ndpi"]["expression"]

differenceName = bandCombinations["difference"]["name"]
quotientName = bandCombinations["quotient"]["name"]

In [ ]:
# `VVVVD`
VVVVD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", b1))

# `VHVHD`
VHVHD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(vh),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(differenceName.replace("b1", vh).replace("b2", vh))

# `VVVHD`
VVVHD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", vh))

# `PRE_VVVHQ`
PRE_VVVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", vh))

# `POST_VVVHQ`
POST_VVVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", vh))

# `VVVVQ`
VVVVQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", b1))

# `VHVHQ`
VHVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(vh),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", vh).replace("b2", vh))

# `VVVHQ`
VVVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", vh))

# `PRE_NDPI`
preNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename("PRE_NDPI")

# `POST_NDPI`
postNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(vh)
    }
  })  \
  .rename("POST_NDPI")

# `NDPID`
NDPID = postNDPI.subtract(preNDPI).rename("NDPID")

# `NDPIQ`
NDPIQ = postNDPI.divide(preNDPI).rename("NDPIQ")

# Rename S1-GRD raster bands using a regular expression.
preEventSamplesSource = preEventRaster.regexpRename("V", "PRE_V", False)
postEventSamplesSource = postEventRaster.regexpRename("V", "POST_V", False)

# Add bands to both pre- & post- event rasters.
preEventRaster = preEventRaster.addBands([PRE_VVVHQ])
postEventRaster = postEventRaster.addBands([POST_VVVHQ])

# Generate a composite with the essential bands for classification.
raster = preEventSamplesSource.addBands([
  postEventSamplesSource, VVVVD, VHVHD, VVVHD, VVVVQ,
  VHVHQ, VVVHQ, preNDPI, postNDPI, NDPID, NDPIQ
])

Raster classification

In [ ]:
# Construct a RF classifier from the decision tree feature collection.
trees = ee.FeatureCollection(classifierIdentifier).aggregate_array("tree")
classifier = ee.Classifier.decisionTreeEnsemble(trees)

# Perform classification.
classified = raster.select(classifierFeatures).classify(classifier)

# **7) Console**

In [ ]:
print("*dates*")
print(f"\t event date: `{eventDate.format('YYYY-MM-DD').getInfo()}`")
print(f"\t acquisition date: `{acquisition.format('YYYY-MM-DD').getInfo()}`")
print(f"\t difference (in days): `{acquisition.difference(eventDate, 'days').getInfo()}`")

*dates*
	 event date: `2018-03-86`
	 acquisition date: `2023-09-250`
	 difference (in days): `1990.0584722222222`


# **8) Export Tasks**

Submit tasks.

In [ ]:
exportTask = ee.batch.Export.image.toAsset(**{
  "description": rasterIdentifier,
  "maxPixels": 1e13,
  "image": classified,
  "scale": projectionScale,
  "crs": projectionCRS,
  "region": caseConfig["area_of_interest"].geometry(),
  "assetId": "/".join([destinationFolder, rasterIdentifier])
})

exportTask.start()

Monitor tasks.

In [ ]:
export_tasks_viewer([exportTask.id])

Task_Id                   Task_State    Task_Type       Task_Attempt  Task_Description                               Queue_Time    Execution_Time    Completion_Time  Error_Message
WPRTRDKCUBIIYWQ5X77EZ5ER  COMPLETED     EXPORT_IMAGE               1  041299_04E8BD_E9C5_050224_060B99_D80F_slope        13.503           925.756            939.259


-End of Notebook-